In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('parkinsons_data.csv')
data.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,0.02182,0.03130,0.02971,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,0.03134,0.04518,0.04368,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,0.02757,0.03858,0.03590,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,0.02924,0.04005,0.03772,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,0.03490,0.04825,0.04465,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [3]:
del data['name']
data.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,0.02182,0.03130,0.02971,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,0.03134,0.04518,0.04368,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,0.02757,0.03858,0.03590,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,0.02924,0.04005,0.03772,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,0.03490,0.04825,0.04465,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [4]:
X = data.loc[:,data.columns!='status']
Y = data['status']

In [5]:
from imblearn.over_sampling import RandomOverSampler

os = RandomOverSampler(random_state=0)

x,y = os.fit_resample(X,Y)

In [6]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0,test_size=0.2)

In [7]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(x_train)
x_train_scaled = ss.transform(x_train)
x_test_scaled = ss.transform(x_test)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Dropout

In [9]:
clf = Sequential()

In [10]:
# Adding the input layer and the first hidden layer
clf.add(Dense(units = 10, kernel_initializer = 'he_uniform',activation='relu',input_dim = 22))
# Adding the second hidden layer
clf.add(Dense(units = 12, kernel_initializer = 'he_uniform',activation='relu'))
# Adding the output layer
clf.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

In [11]:
# Compiling the ANN
clf.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
# Fitting the ANN to the Training set
model_history=clf.fit(x_train_scaled, y_train, batch_size=5, epochs = 100)

Epoch 1/100
47/47 [==============================] - 1s 2ms/step - loss: 0.5967 - accuracy: 0.6298
Epoch 2/100
47/47 [==============================] - 0s 2ms/step - loss: 0.5272 - accuracy: 0.7660
Epoch 3/100
47/47 [==============================] - 0s 2ms/step - loss: 0.4675 - accuracy: 0.8128
Epoch 4/100
47/47 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.8255
Epoch 5/100
47/47 [==============================] - 0s 2ms/step - loss: 0.3902 - accuracy: 0.8383
Epoch 6/100
47/47 [==============================] - 0s 2ms/step - loss: 0.3686 - accuracy: 0.8383
Epoch 7/100
47/47 [==============================] - 0s 2ms/step - loss: 0.3439 - accuracy: 0.8511
Epoch 8/100
47/47 [==============================] - 0s 2ms/step - loss: 0.3259 - accuracy: 0.8553
Epoch 9/100
47/47 [==============================] - 0s 1ms/step - loss: 0.3096 - accuracy: 0.8681
Epoch 10/100
47/47 [==============================] - 0s 2ms/step - loss: 0.2947 - accuracy: 0.8851
Epoch 11/

In [13]:
def prob(a):
  a_prob = []
  for i in a:
    if(i>0.5):
      a_prob.append(1)
    else:
      a_prob.append(0)
  return a_prob

In [14]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

y_pred = prob(clf.predict(x_test_scaled))

In [15]:
confusion_matrix(y_test,y_pred)

array([[32,  0],
       [ 2, 25]])

In [16]:
accuracy_score(y_test,y_pred)*100

96.61016949152543

In [17]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        32
           1       1.00      0.93      0.96        27

    accuracy                           0.97        59
   macro avg       0.97      0.96      0.97        59
weighted avg       0.97      0.97      0.97        59

